In [29]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score , mean_absolute_error

In [30]:
train_df = pd.read_csv('data.csv')
test_df = pd.read_csv('test.csv')

In [31]:


def generate_index(list):
    return {value: key for key, value in enumerate(list, 1)}


def dropExtraColumns(data):
    columns = data.columns
    data.drop(columns[0], axis=1, inplace=True)
    data.drop('Temperature in Montreal during episode', axis=1, inplace=True)
    data.drop('Name of episode', axis=1, inplace=True)
    data.drop('Name of show', axis=1, inplace=True)
    
    

def mapping(data):
    binary_map = {"Yes": 1, "No": 0}
    episode_map = generate_index(data['Episode'].unique())
    station_map = generate_index(data['Station'].unique())
    season_map = generate_index(data['Season'].unique())
    channel_map = generate_index(data['Channel Type'].unique())
    year_map = generate_index(data["Year"].unique())
    week_map = generate_index(data['Day of week'].unique())
    genre_map = generate_index(data['Genre'].unique())
    date_map = generate_index(data['Date'].unique())

    data['Station'] = data['Station'].map(station_map)
    data['Episode'] = data['Episode'].map(episode_map)
    data['Channel Type'] = data['Channel Type'].map(channel_map)
    data['Season'] = data['Season'].map(season_map)
    data['Year'] = data["Year"].map(year_map)
    data['Day of week'] = data['Day of week'].map(week_map)
    data['Genre'] = data['Genre'].map(genre_map)
    data['First time or rerun'] = data['First time or rerun'].map(binary_map)
    data['Movie?'] = data['Movie?'].map(binary_map)
    data['# of episode in the season'] = data['# of episode in the season'].map(binary_map)
    data['Game of the Canadiens during episode?'] = data['Game of the Canadiens during episode?'].map(binary_map)
    data['Date'] = data['Date'].map(date_map)
    

def processTime(data):
    data['Start_time'] = pd.to_datetime(data['Start_time'])
    data['Start_minutes'] = data['Start_time'].dt.hour * 60 + data['Start_time'].dt.minute + data[
        'Start_time'].dt.second / 60
    data['End_time'] = pd.to_datetime(data['End_time'])
    data['End_minutes'] = data['End_time'].dt.hour * 60 + data['End_time'].dt.minute + data[
        'End_time'].dt.second / 60
    data.drop('Start_time', axis=1, inplace=True)
    data.drop('End_time', axis=1, inplace=True)

In [ ]:
dropExtraColumns(train_df)
dropExtraColumns(test_df)

In [33]:
mapping(test_df)
mapping(train_df)

In [34]:
train_df = train_df.dropna(axis = 0)
test_df = test_df.dropna(axis= 0)

Episode                                  0
Station                                  0
Channel Type                             0
Season                                   0
Year                                     0
Date                                     0
Day of week                              0
Start_time                               0
End_time                                 0
Length                                   0
Genre                                    0
First time or rerun                      0
# of episode in the season               0
Movie?                                   0
Game of the Canadiens during episode?    0
Market Share_total                       0
dtype: int64

In [35]:
processTime(train_df)
processTime(test_df)

In [36]:
train_df

,Episode,Station,Channel Type,Season,Year,Date,Day of week,Length,Genre,First time or rerun,# of episode in the season,Movie?,Game of the Canadiens during episode?,Market Share_total,Start_minutes,End_minutes
0,1,1,1,1,1,1,1,8,1,0,1,0,0,0.9,360.0,480.0
1,2,1,1,1,1,1,1,2,2,0,1,0,0,0.5,480.0,510.0
2,3,1,1,1,1,1,1,2,3,0,1,0,0,0.3,510.0,540.0
3,4,1,1,1,1,1,1,4,4,0,1,0,0,1.7,540.0,600.0
4,5,1,1,1,1,1,1,2,5,0,1,0,0,2.2,600.0,630.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
616651,1276,20,2,2,4,877,2,2,7,0,1,0,0,0.0,480.0,510.0
616652,2580,20,2,2,4,877,2,2,7,0,1,0,0,1.1,510.0,540.0
616653,6605,20,2,2,4,877,2,4,22,0,1,0,0,0.3,540.0,600.0
616654,6605,20,2,2,4,877,2,4,22,0,1,0,0,0.4,600.0,660.0


In [37]:
test_df

,Episode,Station,Channel Type,Season,Year,Date,Day of week,Length,Genre,First time or rerun,# of episode in the season,Movie?,Game of the Canadiens during episode?,Start_minutes,End_minutes
0,1,1,1,1,1,1,1,2,1,0,1,0,0,690.0,720.0
1,2,1,1,1,1,1,1,2,1,0,1,0,0,720.0,750.0
2,3,1,1,1,1,1,1,2,1,0,1,0,0,750.0,780.0
3,4,1,1,1,1,1,1,4,2,1,1,0,0,780.0,840.0
4,5,1,1,1,1,1,1,4,2,0,1,0,0,840.0,900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154160,647,3,1,3,1,216,6,4,8,0,1,0,0,1080.0,1140.0
154161,646,3,1,3,1,216,6,4,11,0,1,0,0,1140.0,1200.0
154162,454,3,1,3,1,216,6,6,9,0,1,1,0,1200.0,1290.0
154163,1774,3,1,3,1,216,6,10,9,0,1,1,0,1290.0,0.0


In [38]:
y_train = train_df["Market Share_total"]
x_train = train_df.drop('Market Share_total', axis = 1)
x_test = test_df

In [42]:
from sklearn.ensemble import RandomForestRegressor
predictor = RandomForestRegressor()
